In [31]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from collections import Counter
from torch.utils.data import DataLoader, WeightedRandomSampler, TensorDataset
from sklearn.utils.class_weight import compute_class_weight
import torch.optim as optim
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, accuracy_score
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
base_dir = "SKAB"
SEQ_LENGTH = 8

In [4]:
# Load data from directories
def load_data(base_dir):
    data = []
    labels = []
    for folder in ["anomaly-free", "other", "valve1", "valve2"]:
        folder_path = os.path.join(base_dir, folder)
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            # Load the CSV file and parse the 'datetime' column
            df = pd.read_csv(file_path, sep=";", parse_dates=["datetime"])
            
            # Assign labels based on the folder and check if 'anomaly' column exists
            if folder == "anomaly-free":
                # If the folder is 'anomaly-free', assign a 0 label for all rows
                data.append(df)
                labels.extend([0] * len(df))  # Label all rows as 0 (no anomaly)
            else:
                data.append(df)  # Add to data list for other folders as well
                
                # For other folders, check for the 'anomaly' column
                if "anomaly" in df.columns:
                    labels.extend(df["anomaly"].tolist())  # Use 'anomaly' column for labeling
    
    # Concatenate the data from all files
    df_combined = pd.concat(data, ignore_index=True)
    
    # Sort the concatenated dataframe by 'datetime' to maintain the temporal order
    df_combined = df_combined.sort_values(by="datetime")
    
    return df_combined, labels

In [5]:
# Load and preprocess data
all_data_df, data_labels = load_data(base_dir)

In [6]:
print(len(all_data_df), len(data_labels))

46860 46860


In [7]:
all_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46860 entries, 0 to 44667
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   datetime             46860 non-null  datetime64[ns]
 1   Accelerometer1RMS    46860 non-null  float64       
 2   Accelerometer2RMS    46860 non-null  float64       
 3   Current              46860 non-null  float64       
 4   Pressure             46860 non-null  float64       
 5   Temperature          46860 non-null  float64       
 6   Thermocouple         46860 non-null  float64       
 7   Voltage              46860 non-null  float64       
 8   Volume Flow RateRMS  46860 non-null  float64       
 9   anomaly              37459 non-null  float64       
 10  changepoint          37459 non-null  float64       
dtypes: datetime64[ns](1), float64(10)
memory usage: 4.3 MB


In [7]:
# Above shows that there are no missing entries in the rows. The "anomaly" and "changepoint" columns need to be
# removed due to data leakage.

In [8]:
# Understand class imbalance
label_counts = Counter(data_labels)

# Display the counts for each class
for label, count in label_counts.items():
    print(f"Class {label}: {count} samples")

Class 0: 33619 samples
Class 1.0: 13241 samples


In [9]:
def create_sequences(data, labels, sequence_length):    
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])
        y.append(labels[i + sequence_length])  # Target is the label at the next time step
    return np.array(X), np.array(y)

In [11]:
def preprocess_data(df, labels, sequence_length=8):
    # Exclude columns that could cause data leakage (e.g., "anomaly" or "changepoint")
    columns_to_exclude = ["anomaly", "changepoint"]
    for col in columns_to_exclude:
        if col in df.columns:
            df = df.drop(columns=[col])
    
    # Only want to use the MinMaxScaler on numerical data, not the datetime
    numeric_data = df.select_dtypes(include=["float64", "int64"])
    
    # Scale the numerical data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(numeric_data)
    
    # Create sequences and shuffle with labels
    X, y = create_sequences(scaled_data, labels, sequence_length)
    X, y = shuffle(X, y, random_state=42)

    # Split with stratification
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.4, stratify=y, random_state=42
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
    )
    return X_train, X_val, X_test, y_train, y_val, y_test

In [12]:
X_train, X_val, X_test, y_train, y_val, y_test = preprocess_data(all_data_df, data_labels, SEQ_LENGTH)

# Outputs for inspection
print("Train set size:", len(X_train))
print("Validation set size:", len(X_val))
print("Test set size:", len(X_test))

Train set size: 28111
Validation set size: 9370
Test set size: 9371


In [13]:
def create_dataloaders(X_train, X_val, X_test, y_train, y_val, y_test, batch_size=32):
    # Convert the data and labels into PyTorch tensors
    train_data = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
    val_data = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long))
    test_data = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))
    
    # Compute class weights for CrossEntropyLoss
    class_weights = compute_class_weight(
        class_weight='balanced', 
        classes=np.array([0, 1]), 
        y=y_train
    )
    class_weights = torch.tensor(class_weights, dtype=torch.float32)
    
    # Create the WeightedRandomSampler for the training data
    # The sampler will use the weights calculated from the class distribution
    class_sample_count = [len(torch.nonzero(torch.tensor(y_train) == i)) for i in [0, 1]]
    weights = [class_weights[int(label)] for label in y_train]
    sampler = WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

    # Create DataLoader for training, validation, and test sets
    train_loader = DataLoader(train_data, batch_size=batch_size, sampler=sampler)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader, class_weights

In [14]:
# Define the GRU model
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, dropout=0.2, bidirectional=True):
        super(GRUModel, self).__init__()
        self.model_type = "GRU"
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        
        # GRU layer with bidirectional option
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,  # Dropout only applied if num_layers > 1
            bidirectional=bidirectional
        )
        
        # Batch normalization for the GRU output
        self.batch_norm = nn.BatchNorm1d(hidden_size * 2 if bidirectional else hidden_size)
        
        # Fully connected layer mapping GRU output to target output size
        self.fc = nn.Linear(hidden_size * 2 if bidirectional else hidden_size, output_size)
        
        # Dropout for regularization before fully connected layer
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Determine batch size dynamically
        batch_size = x.size(0)
        
        # Initialize hidden state
        h0 = torch.zeros(
            self.num_layers * (2 if self.bidirectional else 1),  # Double layers for bidirectional
            batch_size,
            self.hidden_size
        ).to(x.device)  # Ensure the hidden state is on the same device as input

        # GRU forward pass
        out, _ = self.gru(x, h0)
        
        # Apply batch normalization to the last time step output
        out_last = out[:, -1, :]  # Get the output of the last time step
        out_last = self.batch_norm(out_last)
    
        # Apply dropout and fully connected layer
        out_last = self.dropout(out_last)
        out = self.fc(out_last)
        
        return out

In [32]:
# Define hyperparameters
input_size = X_train.shape[1]  # Number of features
hidden_size = 64  # Hidden state dimension
output_size = 2  # Number of classes (normal, anomaly)
num_layers = 2
num_epochs = 25
batch_size = 64

# Create the dataloaders and get class weights for loss function
train_loader, val_loader, test_loader, class_weights = create_dataloaders(X_train, X_val, X_test, 
                                                                          y_train, y_val, y_test, batch_size=batch_size)

In [30]:
# Training loop
model = GRUModel(input_size, hidden_size, output_size, num_layers, dropout=0.3).to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=0.001)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='min',       # Reduce the LR when the validation loss stops improving
    factor=0.5,       # Reduce the LR by half
    patience=4,       # Wait for 5 epochs before reducing LR
    verbose=True,     # Print a message when the LR is updated
    min_lr=1e-6       # Minimum learning rate
)
criterion = nn.CrossEntropyLoss(weight=class_weights.clone().detach().to(device))

best_val_loss = float('inf')
best_model_path = "best_model.pth"

# Add tracking for F1 score
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct, total = 0, 0
    all_preds, all_targets = [], []

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track metrics
        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == targets).sum().item()
        total += targets.size(0)
        
        # Collect predictions and targets for F1 score
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())

    train_acc = correct / total
    train_f1 = f1_score(all_targets, all_preds, average='weighted')  # Use 'weighted' for class imbalance
    print(f"\nEpoch {epoch+1}/{num_epochs}, Train Accuracy: {train_acc:.4f}, F1 Score: {train_f1:.4f}")

    # Validation loop
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    val_preds, val_targets = [], []
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == targets).sum().item()
            val_total += targets.size(0)

            # Collect predictions and targets for F1 score
            val_preds.extend(predicted.cpu().numpy())
            val_targets.extend(targets.cpu().numpy())

    val_accuracy = 100 * val_correct / val_total
    avg_val_loss = val_loss / len(val_loader)
    val_f1 = f1_score(val_targets, val_preds, average='weighted')  # Use 'weighted' for class imbalance
    print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.2f}, F1 Score: {val_f1:.4f}")

    # Save the model if validation loss has improved
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), best_model_path)
        print(f"Model saved with validation loss: {avg_val_loss:.4f}")

    # Step the scheduler after validation
    scheduler.step(avg_val_loss)  # This will adjust the learning rate based on validation loss

/home/abey/.local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(



Epoch 1/25, Train Accuracy: 0.5508, F1 Score: 0.4888
Validation Loss: 0.7646, Accuracy: 30.22, F1 Score: 0.1650
Model saved with validation loss: 0.7646

Epoch 2/25, Train Accuracy: 0.6084, F1 Score: 0.5523
Validation Loss: 0.7086, Accuracy: 46.01, F1 Score: 0.4293
Model saved with validation loss: 0.7086

Epoch 3/25, Train Accuracy: 0.6262, F1 Score: 0.5729
Validation Loss: 0.6550, Accuracy: 49.96, F1 Score: 0.4858
Model saved with validation loss: 0.6550

Epoch 4/25, Train Accuracy: 0.6266, F1 Score: 0.5761
Validation Loss: 0.6811, Accuracy: 45.74, F1 Score: 0.4256

Epoch 5/25, Train Accuracy: 0.6542, F1 Score: 0.6219
Validation Loss: 0.5264, Accuracy: 65.78, F1 Score: 0.6733
Model saved with validation loss: 0.5264

Epoch 6/25, Train Accuracy: 0.6766, F1 Score: 0.6527
Validation Loss: 0.5083, Accuracy: 67.67, F1 Score: 0.6917
Model saved with validation loss: 0.5083

Epoch 7/25, Train Accuracy: 0.6838, F1 Score: 0.6629
Validation Loss: 0.5075, Accuracy: 70.58, F1 Score: 0.7202
Mode

In [33]:
# Load the best model weights before validation
model.load_state_dict(torch.load(best_model_path, weights_only=True))
print("Loaded best model weights from validation.")

test_loss = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        test_loss += loss.item()

        # Predictions and labels
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Compute metrics
test_accuracy = accuracy_score(all_labels, all_preds)
test_f1 = f1_score(all_labels, all_preds, average="weighted")

print(f"Test Loss: {test_loss / len(test_loader):.4f}")
print(f"Test Accuracy: {test_accuracy:.2%}")
print(f"Test F1 Score: {test_f1:.4f}")

Loaded best model weights from validation.
Test Loss: 0.5344
Test Accuracy: 72.49%
Test F1 Score: 0.7386
